# Title: Locating the Ori of Bacteria 

# Author: Surur Khan 



# Why Try To Locate the Ori?

1. Gene Therapy - intentionally infect a patient who lacks a crucial gene with a viral vector containing an artificial gene that encodes a therapeutic protein. Once inside the cell, the vector replicates and eventually produces many copies of the therapeutic protein, which in turn treats the patient’s disease. To ensure that the vector actually replicates inside the cell, biologists must know where ori is in the vector’s genome and ensure that the genetic manipulations that they perform do not affect it(This usage is also referred to as Genetic Engineering). The scripts were tested against an E.Coli Genome dataset from the NCBI and yielded significant results for finding it's ori by analyzing the GC content changes and implications of a potential 9mer DnaA box due to a conservative sequence found in a relatively short segment of the Genome.

2. Ensure Genetic Manipulations do not affect overall cell functions 

# What I Test on

The package Bioezy (v 0.0.4) contains several scripts for analyzing genomic data, mainly to locate replication origins. 

## For this analysis, the bacterium **E. Coli (Vibrio Cholerae)** is used. For Credibility, the nucleotide sequence appearing in the ori of E. Coli was predetermined as follows:

atcaatgatcaacgtaagcttctaagcatgatcaaggtgctcacacagtttatccacaac
ctgagtggatgacatcaagataggtcgttgtatctccttcctctcgtactctcatgacca
cggaaagatgatcaagagaggatgatttcttggccatatcgcaatgaatacttgtgactt
gtgcttccaattgacatcttcagcgccatattgcgctggccaaggtgacggagcgggatt
acgaaagcatgatcatggctgttgttctgtttatcttgttttgactgagacttgttagga
tagacggtttttcatcactgactagccaaagccttactctgcctgacatcgaccgtaaat
tgataatgaatttacatgcttccgcgacgatttacctcttgatcatcgatccgattgaag
atcttcaattgttaattctcttgcctcgactcatagccatgatgagctcttgatcatgtt
tccttaaccctctattttttacggaagaatgatcaagctgctgctcttgatcatcgtttc

hereinafter referred to as **vibrio_cholerae_ori.txt**

The actual Genome spans 1,108,250 nucleotides long, which can be found at:

[E.Coli_Genome](https://bioinformaticsalgorithms.com/data/realdatasets/Replication/Vibrio_cholerae.txt)

I will not be analyzing this, rather, I will be using a segment for the sake of runtime, denoted **"vibrio_cholerae_genome_short"**

# What I Look for

There is no shortage of agreement that DNA replication occurs in all cells as it is the nature of living organisms. This however begs the question, how does the cell know where to begin replication in this short region within the otherwise huge genome? It is known that a protein that initiates replication, **DnaA** by binding to a short segment in the ori, **DnaA Box**. My goal to locate these DnaA Boxes is based on a theorum by William Legrand in Edgar Allan Poe's story "The Gold-Bug":

_Assuming DNA is a language of its own, locate frequent words within the ori because for various biological processes, certain nucleotide strings often appear surprisingly often in small regions of the genome_
This is often because certain proteins can only bind to DNA if a specific string of nucleotides is present, and if there are more occurrences of the string, then it is more likely that binding will successfully occur. (It is also less likely that a mutation will disrupt the binding process.)

## Hence, I will extensively analyze the E. Coli Genome to look for its Ori by locating conservative sequences in a relatively short segment of the whole genome

In [29]:
from bioezy import bzy
import os

In [2]:
print(os.getcwd())

/home/sururkhan/Python Code(wsl)/bioezypkg/Vibrio_Cholerae_Analysis


In [10]:
#Important note: frequent words is not usually used for finding frequent words because it has a runtime of O(n^2);  (|Text| − k + 1) · (|Text| − k + 1) · k == |Text|^2 · k
#On specific ori region
with open("vibrio_cholerae_ori.txt","r") as input_file:
    vc_ori = ''.join(line.rstrip() for line in input_file).lower() #manipulation to make it 1 consecutive string
    input_file.close()
print(vc_ori)
    

atcaatgatcaacgtaagcttctaagcatgatcaaggtgctcacacagtttatccacaacctgagtggatgacatcaagataggtcgttgtatctccttcctctcgtactctcatgaccacggaaagatgatcaagagaggatgatttcttggccatatcgcaatgaatacttgtgacttgtgcttccaattgacatcttcagcgccatattgcgctggccaaggtgacggagcgggattacgaaagcatgatcatggctgttgttctgtttatcttgttttgactgagacttgttaggatagacggtttttcatcactgactagccaaagccttactctgcctgacatcgaccgtaaattgataatgaatttacatgcttccgcgacgatttacctcttgatcatcgatccgattgaagatcttcaattgttaattctcttgcctcgactcatagccatgatgagctcttgatcatgtttccttaaccctctattttttacggaagaatgatcaagctgctgctcttgatcatcgtttc


In [3]:
FreqWords = bzy.frequent_words(vc_ori,9) 
for kmer in FreqWords:
    print(f' 9mer: {kmer} appeared {bzy.pattern_count(vc_ori,kmer)} times')



 9mer: atgatcaag appeared 3 times
 9mer: ctcttgatc appeared 3 times
 9mer: tcttgatca appeared 3 times
 9mer: cttgatcat appeared 3 times


# Significance

We highlight a most frequent 9-mer instead of using some other value of k because experiments have revealed that bacterial DnaA boxes are usually nine nucleotides long. The probability that there exists a 9-mer appearing three or more times in a randomly generated DNA string of length 500 is approximately 1/1300.

## Important observation: atgatcaag and cttgatcat are reverse complements of each other, hence **now we must account for the reverse compliment of each frequent 9mer appearing as well!**. We can strongly deduce that DnaA does not need to bind to specifically bind to either the 5'-> 3' forward(lagging) strand nor the 3' -> 5' reverse(leading) strand!

Before concluding that we have found the DnaA box of Vibrio cholerae, we'll check if there are other short regions in the Vibrio cholerae genome exhibiting multiple occurrences of atgatcaag or cttgatcat. Maybe these strings occur as repeats throughout the entire Vibrio cholerae genome, rather than just in the ori region. We will use the pattern matching function to determine this.


In [4]:
print(bzy.pattern_matching("atgatcaag",vc_ori)) #Locations of atgatcaag in ori
print(bzy.pattern_matching("cttgatcat",vc_ori))

[27, 127, 508]
[397, 468, 525]


In [43]:
"""
TESTING ON SHORT GENOME FOR ATGATCAAG
"""

with open("vibrio_cholerae_genome_short.txt","r") as input_file:
    vc_genome_short = ''.join(line.rstrip() for line in input_file).lower()
    input_file.close()

#print(vc_genome)
print(bzy.pattern_matching("atgatcaag",vc_genome_short))

[116556, 149355, 151913, 152013, 152394, 186189, 194276, 200076, 224527, 307692, 479770, 610980, 653338, 679985, 768828, 878903, 985368]


# Significance

## After solving the Pattern Matching Problem, we discover that ATGATCAAG appears 17 times in the starting positions labelled above. With the exception of the three occurrences of ATGATCAAG in ori at starting positions 151913, 152013, and 152394, no other instances of ATGATCAAG form clumps, i.e., appear close to each other in a small region of the genome.

However, a definite conclusion cannot be drawn without checking if it even appears in known ori regions from other bacteria. The case may be that the clumps of my 9mers are statistically insignificant and have nothing to do with replication. 

## Now, the bacterium **Thermotoga petrophila**, an extremophile will be examined to locate it's ori. A potential discovery may be that different bacterium have different oris.

In [6]:
with open("thermotoga_petrophila.txt","r") as input_file:
    tp_ori = ''.join(line.rstrip() for line in input_file).lower()
    input_file.close()

FreqWords = bzy.frequent_words(tp_ori,9) 
for kmer in FreqWords:
    print(f' 9mer: {kmer} appeared {bzy.pattern_count(tp_ori,kmer)} times')

print(bzy.pattern_matching("acctaccac",tp_ori))
print("Other frequent 9mers that appear more than 2 but less than 5 times: AACCTACCA 3 AAACCTACC 3 ACCTACCAC 5 CCTACCACC 3 GGTAGGTTT 3 TGGTAGGTT 3")

 9mer: acctaccac appeared 5 times
[184, 379, 390, 401, 479]
Other frequent 9mers that appear more than 2 but less than 5 times: AACCTACCA 3 AAACCTACC 3 ACCTACCAC 5 CCTACCACC 3 GGTAGGTTT 3 TGGTAGGTT 3


## It is worth noting that AACCTACCA and TGGTAGGTT are reverse complements, as are AAACCTACC and GGTAGGTTT 

# The Caveat

 Searching for “clumps” of either ATGATCAAG (reverse compliment -CTTGATCAT) or CCTACCACC(reverse compliment - GGTGGTAGG) is unlikely to help, since this new genome may use a completely different DnaA Box.

 Instead of finding clumps of a specific k-mer, try to find every k-mer that forms a clump in the genome, hoping they shed light on the location of ori.

 # The Clump Finding Problem 

 Slide a window of fixed length L along the genome, looking for a region where a k-mer appears several times in short succession. The parameter value L = 500 reflects the typical length of ori in bacterial genomes. Define a k-mer as a "clump" if it appears many times within a short interval of the genome. More formally, given integers L and t, a k-mer Pattern forms an (L, t)-clump inside a (longer) string Genome if there is an interval of Genome of length L in which this k-mer appears at least t times. (This definition assumes that the k-mer completely fits within the interval. This also does not take reverse complements into account yet.)

 eg. TGCA forms a (25,3)-clump in the following Genome:
gatcagcataagggtccC**TGCA**A**TGCA**TGACAAGCC**TGCA**GTtgttttac

## From our previous examples of ori regions, ATGATCAAG forms a (500,3)-clump in the Vibrio cholerae genome, and CCTACCACC forms a (500,3)-clump in the Thermotoga petrophila genome.

In [4]:
clumps = bzy.find_clumps(vc_genome_short,9,500,3)
print(len(clumps))

#Refer to scan_clumps_eff_algo for the true solution (sourced online)

acaatgagg ttcgagctc tcgagctct aaccggctg tgcgcatac gcgcatacg gccatccga cagcgtcta gtctattca ggctatgca ctatgcagg caggctact tggttcgta gtaagaact aagaacttt gaactttag gccttacct gctttagtc ctttagtcg tttagtcgt tagtcgtgg cgatctggg gtgtggctc tggctctcg cgcgtatgg gtatggtct gtctgtcta ttctaaccc aacccgcgc ccgcgctac tctgagtgc caggtctac ggtctactc gtctactcc ctactcctt cctttcggc gcactagtg gccatgggc caacgggca acgggcagc agcattact ctgttctta cttaaacgg ctggttcca ggttccaag ttccaaggc tgcttgtgg cttgtggcc ggccgtact tggtgcact ggtgcactg gtgcactgg ggtcacgca agagcgtgg gagcgtggt gtttcggtc tcggtctgg tggaacgtc
58


## Using the scan_clumps program with **vibrio_cholerae_genome_full**, more than 1600 (500,3)-clumped 9mers were found
The result is futile in the search for DnaA Boxes, as the algorithm will locate ALL frequent kmers in the region, which may occur purely by random chance. A new method must be tested.

# The Analysis of the Cytosine Content and it's relevance

DNA replication is an asymmetric process, the faith of the forward (leading) and reverse (lagging) half strands are different. This is because DNA Polymerase, the enzyme responsible for creating the complementary base sequence against each strand can only run in the 3' -> 5' direction. The Reverse half strand indeed runs in this direction and can hence be replicated continuously, however the same cannot be said for the forward half strand, which runs in the 5' -> 3' direction. 

For replication to occur on a forward half strand, the replication fork has to open by about 2000 nucleotides, allowing DNA Polymerase to replicate the strand backwards towards Ori. When this fork closes, replication terminates until it reopens. Due to the inconsistent replication, _okazaki fragments_ are generated, which are partially replicated strands of nucleotides. As a result, many primers are required to fully replicate a forward strand and DNA ligase must be used to join the strand together. The built strand is made of introns and exons, where the former along with the primers are removed by some molecular splicing process. 

The significance of this stems from the fact that because the forward half strand has to wait for its fork to open for replication to occur, it spends most of its life **single stranded**. Naturally, due to the absence of the strongly covalent G-C bonds that would form in a complete double helix, the nucleotide bases are exposed to much higher mutation potentials. Particularly, **Cytosine has a high likeliness to mutate into Thymine** through **deamination**. As a result, **the forward half strand has a much lower Cytosine content than the reverse half strand**. Considering the ori is made up half by the forward and the other half by the reverse, I anticipate that **half of the genome will have a decreasing Cytosine content, indicating that the forward half strand is being observed**. 

# Overall: 

#G-#C decreasing = reverse half strand (high C low G)

#G-#C increasing = forward half strand (low C high G)

## The analysis is now directed towards the idea that **Ori occurs where the reverse half strand transitions into the forward half strand, hence it will be located where an increasing Cytosine content begins to exponentially decrease**

# Generating a Skew Diagram

 Compute Skewi+1(Genome) from Skewi(Genome) according to the nucleotide in position i of Genome. If this nucleotide is G, then Skewi+1(Genome) = Skewi(Genome) + 1; if this nucleotide is C, then Skewi+1(Genome)= Skewi(Genome) – 1; otherwise, Skewi+1(Genome) = Skewi(Genome).

 eg. CATGGGCATCGGCCATACGCC has skew diagram  0 -1 -1 -1 0 1 2 1 1 1 0 1 2 1 0 0 0 0 -1 0 -1 -2

 ## hence, ori will be found where the skew attains a **minimum**


In [17]:
with open("Ecoli_genome.txt","r") as input_file:
    ecoli_genome = ''.join(line.rstrip() for line in input_file).lower() #manipulation to make it 1 consecutive string
    input_file.close()


In [42]:
# Running bzy.minimum_skew(vc_genome_short) is futile, because the algorithm is not robust enough to work with the million length sequence. It will continuously converge to a local optimum, which is not ideal
#  For demonstration purposes, a sample input will be used
bzy.minimum_skew("TAAAGACTGCCGAGAGGCCAACACGAGTGCTAGAACGAGGGGCGTAAACGCGGGTCCGAT")

#Using a more robust analysis, the ori of Vibrio Cholerae is found at approximately position 3923620 of the genome, representing the global optimum.

[11, 24]

 ## Now, the objective is looking for a hidden message representing a potential DnaA box near this location. Solving the Frequent Words Problem in a window of length 500 starting at position **3923620** reveals no 9-mers (along with their reverse complements) that appear three or more times. Even if we have located ori in E. coli, it appears that the location of the DnaA Box is unanimous. 

 ## However, accounting for mismatches, it is seen that in addition to the three occurrences of ATGATCAAG and three occurrences of its reverse complement CTTGATCAT, the Vibrio cholerae ori contains additional occurrences of ATGATCAAC and CATGATCAT, which differ from ATGATCAAG and CTTGATCAT in only a single nucleotide polymorphism. This is referred to as "Hamming Distance". 

 From this, I developed the function approx_pattern_match to return every location of the occurrence of a pattern within the genome with a particular amount of mismatches. Furthermore, the function approx_pattern_count  returns the number of occurrences of a pattern in the specified genome with a particular amount of mismatches

 **For runtime sake, this portion will be ran on the already identified ori region of e.coli**

In [7]:
 bzy.approx_pattern_match(vc_genome_short,"atgatcaag",2) 
#Note they're 2581 occurrences

[14,
 180,
 702,
 1381,
 1579,
 1715,
 1725,
 1746,
 1884,
 2253,
 2396,
 2408,
 2860,
 2964,
 3989,
 4190,
 4210,
 4555,
 4570,
 4762,
 7668,
 7882,
 8461,
 8677,
 8833,
 8857,
 9225,
 9491,
 10422,
 11315,
 11409,
 11711,
 11744,
 11867,
 12190,
 12343,
 13573,
 13579,
 14173,
 14239,
 15079,
 15406,
 15493,
 15739,
 17139,
 17161,
 18168,
 19038,
 19683,
 19790,
 20066,
 20429,
 20753,
 20872,
 21007,
 21235,
 21685,
 21947,
 22138,
 22283,
 22429,
 22887,
 23139,
 23254,
 23536,
 24263,
 24275,
 24452,
 24836,
 24920,
 25038,
 25566,
 25893,
 25928,
 26118,
 26375,
 26632,
 26844,
 27615,
 27723,
 28706,
 29409,
 30105,
 31325,
 31578,
 32491,
 33170,
 33811,
 33821,
 34031,
 34110,
 34113,
 34299,
 34815,
 35228,
 35246,
 35289,
 36477,
 36609,
 38146,
 39371,
 40348,
 40386,
 40720,
 40795,
 42351,
 43021,
 43067,
 43133,
 43384,
 43987,
 44385,
 44499,
 44548,
 44689,
 45282,
 45628,
 45946,
 46233,
 46683,
 46774,
 47118,
 47124,
 47988,
 48680,
 48917,
 49145,
 49247,
 49679,


In [8]:
bzy.approx_pattern_count("atgatcaag",vc_genome_short,2)

2581

## Caveat: For these scripts, All 4k k-mers Pattern, compute approx_pattern_count for each k-mer Pattern, and then find k-mers with the maximum number of approximate occurrences. This is an inefficient approach, since many of the 4k k-mers should not be considered because neither they nor their mutated versions (with up to d mismatches) appear in Text.

## Instead, the algorithm freq_words_mismatch will be used. It uses a single map that counts the number of times a given string has an approximate match in Text. For a given k-mer substring Pattern of Text, we need to increase 1 to the count of every k-mer that has Hamming distance at most d from Pattern. Note here, the vc_ori is used as oppose to Ecoli_genome.txt. My algorithm's runtime is too slow to work with the million length dataset.


In [11]:
bzy.freq_words_mismatch(vc_ori,9,1)

['Catgatcaa',
 'Gatgatcaa',
 'Aatgatcaa',
 'Tatgatcaa',
 'atgatcaaG',
 'atgatcaaA',
 'atgatcaaT',
 'atgatcaaC',
 'atgatcaGg',
 'atgatcaTg',
 'atgatcaCg',
 'atgatcaAg',
 'cGtgatcat',
 'cCtgatcat',
 'cTtgatcat',
 'cAtgatcat']

# Accounting for reverse compliments as well, it is found that within a window of length 500 starting at position 3923620 (3923620 - 3924120) of the E. coli genome, the experimentally confirmed DnaA box in E. coli (TTATCCACA) is indeed a most frequent 9-mer with 1 mismatch, along with its reverse complement TGTGGATAA

## Possible reason for finding multiple other frequent 9mers when looking for ori: 

1. The other 9-mers are for binding other proteins, e.g. a transcription enzyme.

2. DNA Polymerase looks for more than just one DnaA box, and it will recognize different sequences in different boxes.

3. The actual DnaA box is less than 9 bases.  I see strings of C's or G's in the middle, so maybe the surrounding bases don't matter as much.

4. There are more than one DNA Polymerase in e. coli, and they each have different DnaA boxes, and there is some kind of selective pressure to make sure that they all bind in the same region.

# Complications in locating the ori

1.  Some bacteria have even fewer DnaA boxes than E. coli, making it difficult to identify them.

2. The ter region is often located not directly opposite to ori but may be significantly shifted, resulting in reverse and forward half-strands having substantially different lengths. 

3.  The position of the skew minimum is often only a rough indicator of ori position, which forces researchers to expand their windows when searching for DnaA boxes, bringing in extraneous repeated substrings.

4. Skew Diagrams often converge to local minimums due to different evolutionary factors. For example, the skew diagram for Thermotoga petrophila fluctuates heavily. I deduce that as an extremophile, it is adapted to very hot living conditions. As a result, its DNA has to be more stable and hence have a higher GC percentage.